Read in the data. I personally prefer python so here we go

In [ ]:
import h5py
from scipy.sparse import csr_matrix

# Open the HDF5 file
# Go get the matrix from them, I think you need an application or some sort (but should be easy)
file_path = 'NonEpithelial_Count_matrix.h5'
with h5py.File(file_path, 'r') as f:
    # Load the necessary datasets
    data = f['matrix/data'][()]
    indices = f['matrix/indices'][()]
    indptr = f['matrix/indptr'][()]
    shape = f['matrix/shape'][()]

In [ ]:
with h5py.File(file_path, 'r') as f:
    # Load the necessary datasets
    genes = f['matrix/features/name'][()]
    gene_ID = [str(i).split("'")[1] for i in genes]

In [ ]:
matrix = csr_matrix((data, indices, indptr), shape=[shape[1],shape[0]])
meta_data = pd.read_csv('NonEpithelial_metadata.csv',index_col = 0)
import anndata
adata =anndata.AnnData(X = matrix)
adata.var_names = gene_ID
adata.obs_names = meta_data.index
adata.obs = meta_data
adata.write('synapse.h5ad')

Check the metadata, seems like they got everything

In [ ]:
adata.obs[['sample.ID', 'patient.ID', 'sample.origin','dataset','Gender', 'Age at recruitment', 'Group Stage', 'Stage TNM','Site']]

,sample.ID,patient.ID,sample.origin,dataset,Gender,Age at recruitment,Group Stage,Stage TNM,Site
cell.ID,,,,,,,,,
CRC16_MUX8563_AAACCTGCAAGCCGCT-1,MUX8563,CRC2794,Tumor,CRC-SG1,F,65.0,2,T3N0(0/27)M0,Descending colon
CRC16_MUX8563_AAACCTGTCTCGATGA-1,MUX8563,CRC2794,Tumor,CRC-SG1,F,65.0,2,T3N0(0/27)M0,Descending colon
CRC16_MUX8563_AAACCTGTCTCTGTCG-1,MUX8563,CRC2794,Tumor,CRC-SG1,F,65.0,2,T3N0(0/27)M0,Descending colon
CRC16_MUX8563_AAACGGGCAAGTTAAG-1,MUX8563,CRC2794,Tumor,CRC-SG1,F,65.0,2,T3N0(0/27)M0,Descending colon
CRC16_MUX8563_AAACGGGCAGTTCATG-1,MUX8563,CRC2794,Tumor,CRC-SG1,F,65.0,2,T3N0(0/27)M0,Descending colon
...,...,...,...,...,...,...,...,...,...
KUL5_EXT129_TTTGTCAGTTGGACCC-1,EXT129,SC044,Normal,KUL5,F,80.0,III,T3N1M0,Caecum
KUL5_EXT129_TTTGTCATCAGCATGT-1,EXT129,SC044,Normal,KUL5,F,80.0,III,T3N1M0,Caecum
KUL5_EXT129_TTTGTCATCCATGAGT-1,EXT129,SC044,Normal,KUL5,F,80.0,III,T3N1M0,Caecum


In [ ]:
adata.obs['patient'] = adata.obs['patient.ID']
adata.obs['tissue'] = adata.obs['sample.origin']
adata.obs['gender'] = adata.obs['Gender'] 
adata.obs['age'] = adata.obs['Age at recruitment'] 
adata.obs['site'] = adata.obs['Site']
adata.obs['stage'] = adata.obs['Group Stage']

Basic processing, they have filtered data

In [ ]:
sc.pp.normalize_total(adata, key_added = 'normalization_factors')
sc.pp.log1p(adata)

In [ ]:
sc.pp.highly_variable_genes(adata, flavor='seurat', n_top_genes=3000)
sc.pp.pca(adata, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(adata, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(adata, resolution = 0.8, key_added= 'leiden')                                                 

In [ ]:
# Check where we may got some gdT/IL17 producer and further get them by subclustering
sc.pl.umap(adata,color = ['leiden','TRDC','RORC','IL17A','IL17F'],cmap = my_cmap, vmax = 3, size = 40, legend_loc = 'on data')

Isolate IL17 producers, basically labor work

In [ ]:
prob_gd = adata[adata.obs['leiden'].isin(['7']),:]

In [ ]:
sc.pp.highly_variable_genes(prob_gd, flavor = 'seurat', n_top_genes=3000)
#.var['highly_variable'] = adata.var_names.isin(features)
sc.pp.pca(prob_gd, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(prob_gd, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(prob_gd, resolution = 0.8, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(prob_gd)

In [ ]:
%matplotlib inline
plt.rcParams['axes.linewidth'] = 2
fig = sc.pl.tsne(prob_gd, color = ['leiden','ZBTB16','IKZF2','TRGV4','CD3D','CD3G','CD3E',
                                   'CD247','IFNG','IL17A','TRDC','TRDV1','TRDV2','TRDV3','CD4','CD8A',
                                   'PDCD1','KLRK1','RORC','TRAV1-2'],size = 20, legend_loc = 'on data', palette  = my_palette, ncols = 4, cmap = my_cmap, return_fig = True, legend_fontsize = 'large')
ax = fig.get_axes()
for i in range(0,len(ax)):
    ax[i].xaxis.label.set_fontsize(22)
    ax[i].xaxis.label.set_fontweight('bold')
    ax[i].yaxis.label.set_fontsize(22)
    ax[i].title.set_fontsize(30)
    ax[i].yaxis.label.set_fontweight('bold')
    ax[i].title.set_fontweight('bold')
#plt.savefig('overall_map_withunmatched.png')

In [ ]:
gamma_genesmask = [gene.startswith("TRG") for gene in adata.var_names]
gamma_genes = adata.var_names[gamma_genesmask]
delta_genesmask = [gene.startswith("TRD") for gene in adata.var_names]
delta_genes = adata.var_names[delta_genesmask]

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden', ['4']), resolution = 0.9, key_added= 'leiden1')
sc.pl.tsne(prob_gd, color=['leiden1','CD4','CD8A','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden1', ['6']), resolution = 0.35, key_added= 'leiden2')
sc.pl.tsne(prob_gd, color=['leiden2','CD4','CD8A','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden2', ['3']), resolution = 0.35, key_added= 'leiden3')
sc.pl.tsne(prob_gd, color=['leiden3','CD4','CD8A','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden3', ['1']), resolution = 0.9, key_added= 'leiden4')
sc.pl.tsne(prob_gd, color=['leiden4','CD4','CD8A','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden4', ['4,1']), resolution = 0.5, key_added= 'leiden5')
sc.pl.tsne(prob_gd, color=['leiden5','CD4','CD8A','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G']
}
sc.pl.dotplot(prob_gd,TCR_exp_set,groupby = 'leiden5' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

In [ ]:
gd1 = prob_gd[prob_gd.obs['leiden4'].isin(['1,8','4,0','4,1,0','4,1,1','4,3','4,4','4,5','4,6','4,7']),:]

In [ ]:
sc.pl.tsne(gd1,color = ['leiden4','CD3D','CD3E','CD3G','CD247','TRDC'])

In [ ]:
prob_gd = adata[adata.obs['leiden'].isin(['9']),:]

In [ ]:
sc.pp.highly_variable_genes(prob_gd, flavor = 'seurat', n_top_genes=3000)
#.var['highly_variable'] = adata.var_names.isin(features)
sc.pp.pca(prob_gd, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(prob_gd, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(prob_gd, resolution = 0.8, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(prob_gd)

In [ ]:
%matplotlib inline
plt.rcParams['axes.linewidth'] = 2
fig = sc.pl.tsne(prob_gd, color = ['leiden','ZBTB16','IKZF2','TRDC','CD3D','CD3G','CD3E','CD247','IFNG','IL17A','TRDV1','TRDV2','TRDV3','CD3D','CD3E','CD3G','CD247','PDCD1','KLRK1','RORC','TRAV1-2'],size = 20, legend_loc = 'on data', palette  = my_palette, ncols = 4, cmap = my_cmap, return_fig = True, legend_fontsize = 'large')
ax = fig.get_axes()
for i in range(0,len(ax)):
    ax[i].xaxis.label.set_fontsize(22)
    ax[i].xaxis.label.set_fontweight('bold')
    ax[i].yaxis.label.set_fontsize(22)
    ax[i].title.set_fontsize(30)
    ax[i].yaxis.label.set_fontweight('bold')
    ax[i].title.set_fontweight('bold')
#plt.savefig('overall_map_withunmatched.png')

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden', ['6']), resolution = 0.4, key_added= 'leiden1')
sc.pl.tsne(prob_gd, color=['leiden1','CD3D','CD3E','CD3G','CD247','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden1', ['5']), resolution = 0.5, key_added= 'leiden2')
sc.pl.tsne(prob_gd, color=['leiden2','CD3D','CD3E','CD3G','CD247','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden2', ['3']), resolution = 0.5, key_added= 'leiden3')
sc.pl.tsne(prob_gd, color=['leiden3','CD3D','CD3E','CD3G','CD247','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G']
}



sc.pl.dotplot(prob_gd,TCR_exp_set,groupby = 'leiden3' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

In [ ]:
gd2 = prob_gd[prob_gd.obs['leiden3'].isin(['5,2','6,2','6,3']),:]

In [ ]:
sc.pl.tsne(gd2,color = ['leiden3','CD3D','CD3E','CD3G','CD247','TRDC'])

In [ ]:
prob_gd = adata[adata.obs['leiden'].isin(['4']),:]

In [ ]:
sc.pp.highly_variable_genes(prob_gd, flavor = 'seurat', n_top_genes=3000)
#.var['highly_variable'] = adata.var_names.isin(features)
sc.pp.pca(prob_gd, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(prob_gd, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(prob_gd, resolution = 0.8, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(prob_gd)

In [ ]:
%matplotlib inline
plt.rcParams['axes.linewidth'] = 2
fig = sc.pl.tsne(prob_gd, color = ['leiden','ZBTB16','IKZF2','TRDC','CD3D','CD3G','CD3E',
                                   'CD247',],size = 20, legend_loc = 'on data', palette  = my_palette, ncols = 4, cmap = my_cmap, return_fig = True, legend_fontsize = 'large')
ax = fig.get_axes()
for i in range(0,len(ax)):
    ax[i].xaxis.label.set_fontsize(22)
    ax[i].xaxis.label.set_fontweight('bold')
    ax[i].yaxis.label.set_fontsize(22)
    ax[i].title.set_fontsize(30)
    ax[i].yaxis.label.set_fontweight('bold')
    ax[i].title.set_fontweight('bold')
#plt.savefig('overall_map_withunmatched.png')

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden', ['7']), resolution = 0.5, key_added= 'leiden1')
sc.pl.tsne(prob_gd, color=['leiden1','CD3D','CD3E','CD3G','CD247','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden1', ['1']), resolution = 0.6, key_added= 'leiden2')
sc.pl.tsne(prob_gd, color=['leiden2','CD3D','CD3E','CD3G','CD247','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden2', ['6']), resolution = 0.4, key_added= 'leiden3')
sc.pl.tsne(prob_gd, color=['leiden3','CD3D','CD3E','CD3G','CD247','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G']
}



sc.pl.dotplot(prob_gd,TCR_exp_set,groupby = 'leiden3' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

In [ ]:
gd3 = prob_gd[prob_gd.obs['leiden3'].isin(['1,3','6,3','7,1','7,2','9']),:]

In [ ]:
sc.pl.tsne(gd3,color = ['leiden3','CD3D','CD3E','CD3G','CD247','TRDC'])

In [ ]:
prob_gd = adata[adata.obs['leiden'].isin(['14']),:]

In [ ]:
sc.pp.highly_variable_genes(prob_gd, flavor = 'seurat', n_top_genes=3000)
#.var['highly_variable'] = adata.var_names.isin(features)
sc.pp.pca(prob_gd, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(prob_gd, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(prob_gd, resolution = 0.8, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(prob_gd)

In [ ]:
%matplotlib inline
plt.rcParams['axes.linewidth'] = 2
fig = sc.pl.tsne(prob_gd, color = ['leiden','ZBTB16','IKZF2','TRGV4','CD3D','CD3G','CD3E',
                                   'CD247','TRDC'],size = 50, legend_loc = 'on data', palette  = my_palette, ncols = 4, cmap = my_cmap, return_fig = True, legend_fontsize = 'large')
ax = fig.get_axes()
for i in range(0,len(ax)):
    ax[i].xaxis.label.set_fontsize(22)
    ax[i].xaxis.label.set_fontweight('bold')
    ax[i].yaxis.label.set_fontsize(22)
    ax[i].title.set_fontsize(30)
    ax[i].yaxis.label.set_fontweight('bold')
    ax[i].title.set_fontweight('bold')
#plt.savefig('overall_map_withunmatched.png')

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden', ['7']), resolution = 0.3, key_added= 'leiden1')
sc.pl.tsne(prob_gd, color=['leiden1','CD3D','CD3E','CD3G','CD247','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(prob_gd, restrict_to = ('leiden1', ['3']), resolution = 0.5, key_added= 'leiden2')
sc.pl.tsne(prob_gd, color=['leiden2','CD3D','CD3E','CD3G','CD247','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G']
}



sc.pl.dotplot(prob_gd,TCR_exp_set,groupby = 'leiden2' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

In [ ]:
gd4 = prob_gd[prob_gd.obs['leiden2'].isin(['3,2','3,3','7,1']),:]

In [ ]:
sc.pl.tsne(gd4,color = ['leiden2','CD3D','CD3E','CD3G','CD247','TRDC'])

In [ ]:
adata_gd = gd1.concatenate(gd2,gd3,gd4, batch_key = 'original_cluster', batch_categories=['7','9','4','14'],join = 'outer',fill_value=0)

In [ ]:
sc.pp.highly_variable_genes(adata_gd, flavor = 'seurat', n_top_genes=3000)
#.var['highly_variable'] = adata.var_names.isin(features)
sc.pp.pca(adata_gd, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(adata_gd, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(adata_gd, resolution = 0.8, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(adata_gd)

In [ ]:
%matplotlib inline
plt.rcParams['axes.linewidth'] = 2
fig = sc.pl.tsne(adata_gd, color = ['original_cluster','leiden','ZBTB16','IKZF2','TRGV4','CD3D','CD3G','CD3E',
                                   'CD247','IFNG','IL17A','TRDC','TRDV1','TRDV2','TRDV3','CD4',
                                   'CD8A','CD8B','PDCD1','KLRK1','RORC','TRDC','FOXP3','TRAV1-2'],size = 50, legend_loc = 'on data', palette  = my_palette, ncols = 4, cmap = my_cmap, return_fig = True, legend_fontsize = 'large')
ax = fig.get_axes()
for i in range(0,len(ax)):
    ax[i].xaxis.label.set_fontsize(22)
    ax[i].xaxis.label.set_fontweight('bold')
    ax[i].yaxis.label.set_fontsize(22)
    ax[i].title.set_fontsize(30)
    ax[i].yaxis.label.set_fontweight('bold')
    ax[i].title.set_fontweight('bold')
#plt.savefig('overall_map_withunmatched.png')

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(adata_gd, restrict_to = ('leiden', ['4']), resolution = 0.3, key_added= 'leiden1')
sc.pl.tsne(adata_gd, color=['leiden1','CD3E','CD3G','CD3D','TRDC'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
TCR_exp_set ={
"gamma": gamma_genes, "delta": delta_genes, "CD3S": ['CD3E','CD3D','CD3G']
}



sc.pl.dotplot(adata_gd,TCR_exp_set,groupby = 'leiden1' , vmax = 1, swap_axes = False, dot_min =0, dot_max =1,standard_scale = 'var')

In [ ]:
adata_gd_filtered = adata_gd[adata_gd.obs['leiden1'].isin(['4,0','4,2']) == 0,:]

In [ ]:
adata_gd_filtered.write('synapse_gd.h5ad')

Isolate IL17 producers, basically labor work

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['1']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden', ['5']), resolution = 0.4, key_added= 'leiden1')
sc.pl.tsne(potential_17, color=['leiden1','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden1', ['7']), resolution = 0.4, key_added= 'leiden2')
sc.pl.tsne(potential_17, color=['leiden2','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden2', ['2']), resolution = 0.5, key_added= 'leiden3')
sc.pl.tsne(potential_17, color=['leiden3','RORC','IL17A','IL17F'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden3' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_1 = potential_17[potential_17.obs['leiden3'].isin(['2,0','2,1','2,3','2,4','5,2','7,1','7,2','7,3','12']),:]

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['9']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden', ['2']), resolution = 0.9, key_added= 'leiden1')
sc.pl.tsne(potential_17, color=['leiden1','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden1', ['8']), resolution = 0.4, key_added= 'leiden2')
sc.pl.tsne(potential_17, color=['leiden2','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden2', ['2']), resolution = 0.5, key_added= 'leiden3')
sc.pl.tsne(potential_17, color=['leiden3','RORC','IL17A','IL17F'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden2' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_2 = potential_17[potential_17.obs['leiden2'].isin(['2,1','16']),:]

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['8']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_3 = potential_17[potential_17.obs['leiden'].isin(['0']),:]

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['3']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden', ['1']), resolution = 0.9, key_added= 'leiden1')
sc.pl.tsne(potential_17, color=['leiden1','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden1' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_4 = potential_17[potential_17.obs['leiden2'].isin(['2,1','16']),:]

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['4']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden', ['7']), resolution = 0.5, key_added= 'leiden1')
sc.pl.tsne(potential_17, color=['leiden1','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden1', ['4']), resolution = 0.5, key_added= 'leiden2')
sc.pl.tsne(potential_17, color=['leiden2','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden2', ['9']), resolution = 0.5, key_added= 'leiden3')
sc.pl.tsne(potential_17, color=['leiden3','RORC','IL17A','IL17F'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden3', ['3']), resolution = 0.6, key_added= 'leiden4')
sc.pl.tsne(potential_17, color=['leiden4','RORC','IL17A','IL17F'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden4' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_5 = potential_17[potential_17.obs['leiden'].isin(['3,0','3,2','4,1','7,0','7,1','7,3','9,0','9,1']),:]

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['5']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 0.7, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden', ['3']), resolution = 1, key_added= 'leiden1')
sc.pl.tsne(potential_17, color=['leiden1','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden1', ['4']), resolution = 1, key_added= 'leiden2')
sc.pl.tsne(potential_17, color=['leiden2','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden1' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_6 = potential_17[potential_17.obs['leiden1'].isin(['3,7','9']),:]

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['15']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 1, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden', ['3']), resolution = 0.5, key_added= 'leiden1')
sc.pl.tsne(potential_17, color=['leiden1','RORC','IL17A','IL17F'], size = 40, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden1', ['4']), resolution = 0.5, key_added= 'leiden2')
sc.pl.tsne(potential_17, color=['leiden2','RORC','IL17A','IL17F'], size = 20, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden2', ['5']), resolution = 0.5, key_added= 'leiden3')
sc.pl.tsne(potential_17, color=['leiden3','RORC','IL17A','IL17F'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden3', ['10']), resolution = 0.4, key_added= 'leiden4')
sc.pl.tsne(potential_17, color=['leiden4','RORC','IL17A','IL17F'], size = 30, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden4' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_7 = potential_17[potential_17.obs['leiden'].isin(['3,1','3,3','4,0','4,2','5,0','10,2','5,3','4,1']),:]

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['39']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 0.7, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_8 = potential_17[potential_17.obs['leiden'].isin(['4'])==0,:]

In [ ]:
potential_17 = adata[adata.obs['leiden'].isin(['6']),:]

In [ ]:
del potential_17.uns

In [ ]:
sc.pp.highly_variable_genes(potential_17, flavor='seurat', n_top_genes=3000)
sc.pp.pca(potential_17, n_comps=50, use_highly_variable=True, svd_solver='arpack')

In [ ]:
sc.pp.neighbors(potential_17, n_neighbors = 15, n_pcs = 50)
sc.tl.leiden(potential_17, resolution = 0.7, key_added= 'leiden')                                                 

In [ ]:
sc.tl.tsne(potential_17)

In [ ]:
%matplotlib inline
sc.pl.tsne(potential_17, color=['leiden','RORC','IL17A','IL17F'],size = 40,cmap = my_cmap)

In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sc.tl.leiden(potential_17, restrict_to = ('leiden', ['12']), resolution = 0.3, key_added= 'leiden1')
sc.pl.tsne(potential_17, color=['leiden1','RORC','IL17A','IL17F'], size = 40, legend_loc = 'on data', cmap = my_cmap)

In [ ]:
IL17_genesmask = [gene.startswith("IL17") and gene.startswith("IL17R")==0 for gene in potential_17.var_names]
IL17_genes = potential_17.var_names[IL17_genesmask]
IL17_exp_set ={
    'cytokine': IL17_genes, 'TF': 'RORC'
}
sc.pl.dotplot(potential_17,IL17_exp_set,groupby = 'leiden1' , vmax = 1, swap_axes = False, dot_min =0.1, dot_max =1,standard_scale = 'var')

In [ ]:
T17_9 = potential_17[potential_17.obs['leiden1'].isin(['12,0','12,2'])==0,:]

In [ ]:
adata_17 = T17_1.concatenate(T17_2,T17_3,T17_5,T17_6,T17_7,T17_8,T17_9, batch_key = 'original_cluster', join = 'outer',index_unique = None, fill_value=0)